<font color="green">__Сентябрь '2022__</font>

__Сводный блокнот__, объединяющий оба этапа:

- Сбор информации (имя, размер в байтах, расширение, хэш) по файлам из заданной папки;
- Отбор только уникальных в новую папку с сохранением исходной иерархической структуры вложенных папок.

10.01.2022:
- добавлено время создания, последнего доступа и изменения

19.04.2022
-  добавлена возможность обработки длинных имен файлов (с помощью **UNC**), переменная *unc_prefix*

19.09.2022
- учтена обработка UNC в функции по копированию уникальных; удалена зависимость от библиотеки *tqdm*.

## I Подготовительная часть

#### Импорт библиотек

In [ ]:
# built-in libraries, не нужно устанавливать дополнительно
import os
import sys
import logging
import hashlib
import shutil
import platform

from time import localtime, strftime
from datetime import datetime
from pathlib import PurePath

# external libraries, сторонние библиотеки, требуется установка
import pandas as pd
import numpy as np

print(f'Доступные алгоритмы хэширования:\n {hashlib.algorithms_guaranteed}')

In [ ]:
print('Питон здесь:', sys.executable)
print('Версия:', platform.python_version())  # Версия: 3.6.6

In [ ]:
# Проверка версий установленных библиотек
lib_list = [pd, np]

from importlib_metadata import version
for p in lib_list:
    print(f'{p.__name__:12} {version(p.__name__)}')

# проверено для:
# pandas       1.1.5
# numpy        1.19.0
# но может хватить и более ранних версий

 #### Определение функций

In [ ]:
# UNC-префикс, чтобы избежать проблемы длинных имен
unc_prefix = "\\\\?\\"

def calc_hash(file, block_size=2**18):
    """
    Вычисление хэша для файла.
    Размер блока для обработки задан параметром block_size.
    Тип алгоритма хэширования (sha256) задан внутри.
    """
    file_hash = hashlib.sha256() # = Создаем объект с помощью выбранного алгоритма хэширования
    with open(file, 'rb') as f: # Open the file to read it's bytes
        fb = f.read(block_size) # Read from the file. Take in the amount declared above
        while len(fb) > 0: # While there is still data being read from the file
            file_hash.update(fb) # Update the hash
            fb = f.read(block_size) # Read the next block from the file
    return (file_hash.hexdigest()) # Get the hexadecimal digest of the hash

def collect_stat(fld_in):
    """
    Собираем статистику о файлах, обходя папку, полученную на вход -- fld_in.
    10.01.22 - добавил время (создания, последнего доступа, изменения)
    """
    def strftime_loc(timestamp):
        try:
            return strftime("%Y-%m-%d %H:%M:%S", timestamp)
        except:
            return ""
    
    
    time_started = datetime.now()
    print('Начало работы - ', time_started)
    for root, folders, files in os.walk(fld_in):
        for f in files:
            full_path = unc_prefix + os.path.join(root, f)
            # если файл не является ссылкой
            if not os.path.islink(full_path):
                file_size = os.path.getsize(full_path)
                file_ctime = localtime(os.path.getctime(full_path)) # created at
                file_atime = localtime(os.path.getatime(full_path)) # accessed 
                file_mtime = localtime(os.path.getmtime(full_path)) # modified
                fhash = calc_hash(full_path)
            else: # если все же ссылка, то оставляем пустыми
                file_size = None
                file_ctime = None # created at
                file_atime = None # accessed 
                file_mtime = None # modified
                fhash = None
            # расширение можно извлекать в любом случае (и ссылка, и не ссылка)
            ftype = os.path.splitext(full_path)[1][1:]
            msg = "; ".join(list(map(str, [root, full_path, ftype, file_size, fhash])) + 
                            list(map(strftime_loc, [file_ctime, file_atime, file_mtime]))
                           )
            log_hashfiles.info(msg)
    print('Завершено - ', datetime.now())
    print(f'Затрачено времени: {datetime.now() - time_started}')
    
def delete_empty_folders(folder_out):
    """Walk all subfolders of folder_out and
    delete empty (not containing neither files nor folders)"""
    q_deleted = 0
    for item in os.walk(folder_out):
        # берем список подпапок текущей папки
        for subdir in item[1]:
            path_to_subdir = os.path.join(item[0], subdir)
            # если она пустая, то удаляем
            if not os.listdir(path_to_subdir):
                print(path_to_subdir)
                os.rmdir(path_to_subdir)
                q_deleted += 1
    print('Done. Deleted empty folders:', q_deleted)
    return q_deleted


def csv_to_datafr_proc(file_in):
    """
    Читаем путь к csv, обрабатываем,
    возвращаем датафрейм, обогащенный
    10.01.2022 -- добавлены три столбца в заголовок также.
    """
    df = pd.read_csv(file_in,
                     sep=';',
                     encoding='utf8',
                     header=None,
                     names=['Folder', 'Full_path', 'Type', 'Size_bytes', 'Hash', 'ctime', 'atime', 'mtime'])
    try:
        df.Folder = df.Folder.apply(lambda x: str(x)[10:]) # Убираем первые символы 
        df.Type = df.Type.apply(lambda x: str(x).lower())  # Приводим все расширения файлов к нижнему регистру
        df['Drive'] = df.Folder.apply(lambda x: PurePath(str(x)).parts[3] if len(PurePath(str(x)).parts)>3 else "-")
        df['File'] = df.Full_path.apply(lambda x: os.path.basename(str(x)))
        df['dir'] = df['Full_path'].apply(lambda x: os.path.dirname(str(x)).split('/')[-1])
        df['key_fn_hash'] = df['File'].astype('str') + '_' + df['Hash'] # Ключ из двух полей: имя файла_хэш
        df['MegaBytes'] = np.round(df['Size_bytes'] / 1048576, 1)  # Размер файла в Мб, округленный до десятых
    except:
        pass
        print('error')
    print('df.shape:', df.shape)
    return df
          
##################################################################          
# Вспомогательные функции для создания логгера и его сворачивания.
##################################################################
          
def init_logging(log_name, filename, level):
    """
    Создаем логгер для записи событий.
    """
    logger = logging.getLogger(log_name)
    logger.setLevel(level)

    _log_format = f'%(asctime)s: [%(levelname)s]: %(message)s'
    _date_format = '%Y-%m-%d %H:%M:%S'
    file_handler = logging.FileHandler(filename, mode='w', encoding='utf8')
    file_handler.setLevel(level)
    file_handler.setFormatter(logging.Formatter(_log_format, _date_format))
    logger.addHandler(file_handler)
    return logger

def remove_handlers(some_logger):
    """
    Удаляем все хэндлеры из логгера
    """
    for hndl in some_logger.handlers:
        hndl.close()
        some_logger.removeHandler(hndl)

        
########################################################################          
# Функции для 2 части: копирования уникальных по ключу (имя_хэш) файлов
########################################################################

def get_dest_name(source_f, dest_f, full_filename):
    """
    В папке source_f находится структура, которую нужно
    сохранить при копировании в папку dest_f.
    Работаем с полным путем к одному из файлов -- full_filename.
    ---
    Возвращаем новое полное имя в целевой папке с учетом вложенности
    исходного пути.
    Возвращаем None, если переданы несовпадающие логические пути источника
    и находящегося в нем файла.
    """

    try:
        path_sf = PurePath(source_f)
        path_file = PurePath(full_filename)
        second_part = path_file.relative_to(path_sf)
    except:
        # Например, если путь к файлу не дочерний по отношению к папке-источнику
        return None

    return PurePath(dest_f).joinpath(second_part)


def create_dir(dir_name):
    """Создаем папку, если она не существует (и все подпапки на пути к конечной также)
    В случае ошибки, ничего не делаем."""
    try:
        os.makedirs(dir_name)
    except:
        pass

def copy_selected_files(folder_in, check_list, folder_result):
    """Копируем данные из папки folder_in,
    если файл из списка check_list.
    """
    try:
        # проходим по всем подпапкам исходной папки, переданной на вход,
        # и в зависимости от полного пути копируем в целевую или пропускаем
        stdt = datetime.now()
        print('Время начала:', stdt)
        files_copied = 0
        for fp_elem in os.walk(unc_prefix + folder_in):
            for f in fp_elem[2]:
                # формируем полные пути и с ними работаем
                f_name = os.path.join(fp_elem[0], f)
                if f_name in check_list:
                    f_name_dest = get_dest_name(unc_prefix + folder_in,
                                                unc_prefix + folder_result,
                                                f_name)
                    # create directory sub-folder before copying
                    create_dir(os.path.dirname(f_name_dest))
                    shutil.copy2(f_name, f_name_dest)
                    files_copied += 1
        # замеряем выполнение
        enddt = datetime.now()
        print('Время завершения:', enddt)
        print('Затрачено, h:mm:ss:', enddt - stdt)
        print(f'Скопировано файлов: {files_copied}')
    except Exception as exc_err:
        err_msg = 'Ошибка. Данные не скопированы.'
        print(err_msg)
        print(exc_err)

### II Основная часть

### 0. Задаем параметры -- имена папок и файлов.

In [ ]:
# Полный путь к папке, которую анализируем
folder_in = r'C:\Users\user\Documents\20220919_УдалениеДубликатов_Хэш (исправления)\out'

# Для сохранения результатов:
fname = '2022sep19_global'
# Имя текстового файла-лога со статистикой, из которого будет сделан датафрейм
log_name = f'{fname}.txt'
# Имя Excel-файла для сохранения промежуточных результатов (с уникальным ключом ИмяФайла_Хэш)
excel_res_file = f'{fname}.xlsx'
# Путь к папке, куда будут сохраняться отобранные уникальные
f_out = r'C:\Users\user\Documents\20220919_УдалениеДубликатов_Хэш (исправления)\uniq'

### 1. Сбор информации

In [ ]:
%%time

# Инициируем логгер
log_hashfiles = init_logging('log_hashfiles',
                           log_name,
                           logging.INFO)

# Запускаем основную функцию
collect_stat(folder_in)

# Завершаем логгирование
remove_handlers(log_hashfiles)
logging.shutdown()

# для примерно 311 Мб
# Начало работы -  2022-04-06 18:06:25.992508
# Завершено -  2022-04-06 18:13:59.576907
# Затрачено времени: 0:07:33.584399
# Wall time: 7min 33s

### 2. Загружаем в датафрейм, сохраняем в Excel

In [ ]:
%%time

datafr = csv_to_datafr_proc(log_name)
datafr.to_excel(excel_res_file, index=False)

datafr.tail(3)

### 3. Отбираем только уникальные и копируем их в новое место

In [ ]:
# Нашли всего уникальных
datafr['key_fn_hash'].nunique()

In [ ]:
# Составляем таблицу только с уникальными, отобранными по комбинации "Имя файла - Хэш" и встреченными первыми при обходе каталога
df_unique_key = datafr.drop_duplicates(subset=['key_fn_hash'], keep='first')
print(df_unique_key.shape)
df_unique_key

In [ ]:
%%time

# Составляем список имен файлов, которые будем переносить в новую папку
files_to_copy = df_unique_key['Full_path'].apply(lambda x: x.strip()).tolist()
print(f'Считано в список имен файлов: {len(files_to_copy)}')
print(f'Первые 10 значений:\n')
files_to_copy[:10]

In [ ]:
%%time

# копируем в соответствии с полученным списком и полученными ранее данными
copy_selected_files(folder_in, files_to_copy, f_out)

# Пример замера скорости работы
# Время начала: 2022-04-06 18:26:45.999040
# Время завершения: 2022-04-06 18:28:44.798380
# Затрачено, h:mm:ss: 0:01:58.799340
# Скопировано файлов: 11508
# Wall time: 1min 58s

#### *Дополнительно:  удаление пустых вложенных папок (если они встречались) в папке-результате*

In [ ]:
q = 1
while q>0:
    q = delete_empty_folders(f_out)